In [3]:
pip install tensorflow-recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 4.5 MB/s 
     |████████████████████████████████| 578.0 MB 14 kB/s 
     |████████████████████████████████| 1.7 MB 45.3 MB/s 
     |████████████████████████████████| 5.9 MB 45.5 MB/s 
     |████████████████████████████████| 438 kB 59.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstal

In [9]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pprint

In [5]:
data = pd.read_csv('coursesData.csv')

In [11]:
data['id'] = data['id'].map(str)

In [17]:
ratings = tf.data.Dataset.from_tensor_slices(dict(data))
courses = tf.data.Dataset.from_tensor_slices(dict(data[['course']]))

In [18]:
for rate in ratings.take(5).as_numpy_iterator():
  pprint.pprint(rate)

{'course': b'BA in Anthropology', 'id': b'0'}
{'course': b'BA in Archeology', 'id': b'1'}
{'course': b'BA in Archeology and Geography ', 'id': b'2'}
{'course': b'BA in Archeology and History ', 'id': b'3'}
{'course': b'BA in Art and Design', 'id': b'4'}


In [22]:
ratings1 = ratings.map(lambda x:{
    'id':x['id'],
    'course':x['course'],
})
courses1 = courses.map(lambda x:x['course'])

In [20]:
class CoursesModel(tfrs.Model):
  def __init__(self,idModel:tf.keras.Sequential(),courseModel:tf.keras.Sequential(),task:tfrs.tasks.Retrieval()):
    super().__init__()
    self.idModel = idModel
    self.courseModel = courseModel
    self.task  = task

  def compute_loss(self,features,training=False)->tf.Tensor:
    id_embeddings = self.idModel(features['id'])
    course_embeddings = self.courseModel(features['course'])

    return self.task(id_embeddings,course_embeddings)



In [24]:
id_vocabulary = tf.keras.layers.StringLookup()
id_vocabulary.adapt(ratings1.map(lambda x:x['id']))
courses_vocabulary = tf.keras.layers.StringLookup()
courses_vocabulary.adapt(courses1)

In [27]:
idModel = tf.keras.Sequential([
    id_vocabulary,
    tf.keras.layers.Embedding(id_vocabulary.vocab_size(),12)
])
courseModel = tf.keras.Sequential([
    courses_vocabulary,
    tf.keras.layers.Embedding(courses_vocabulary.vocab_size(),12)
])

In [28]:
task = tfrs.tasks.Retrieval(metrics = tfrs.metrics.FactorizedTopK(courses1.batch(10).map(courseModel)))

In [31]:
model = CoursesModel(idModel,courseModel,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [33]:
model.fit(ratings1.batch(10),epochs=20)

Epoch 1/20
12/12 [==============================] - 1s 63ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.4643 - factorized_top_k/top_10_categorical_accuracy: 0.7411 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 18.1302 - regularization_loss: 0.0000e+00 - total_loss: 18.1302
Epoch 2/20
12/12 [==============================] - 1s 59ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.7679 - factorized_top_k/top_10_categorical_accuracy: 0.9286 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 12.1753 - regularization_loss: 0.0000e+00 - total_loss: 12.1753
Epoch 3/20
12/12 [==============================] - 1s 58ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 

In [36]:
index = tfrs.layers.factorized_top_k.BruteForce(model.idModel)
index.index_from_dataset(courses1.batch(20).map(lambda course:(course,model.courseModel(course))))

In [44]:
for i in range(0,112):
  course,titles = index(np.array([str(i)]))
  print(f'course: {data.iloc[i][1]} \n Neighbours are {titles} \n values are {course}')
  print('\n \n')

course: BA in Anthropology 
 Neighbours are [[b'BA in Anthropology' b'Bachelor of Science in Chemistry and Physics'
  b'BA in Political Science and Public Administration '
  b'Master of Arts in Development Studies'
  b'Bachelor of Education in Adult and Community Education '
  b'BA in Public Relations and Advertising'
  b'BSc in Wildlife Science and Conservation'
  b'BSc in Textile Design and Technology'
  b'Bachelor of Science in Chemistry' b'BA in Journalism']] 
 values are [[7.5670085 5.511014  4.1489286 3.809025  3.2065372 2.994997  2.8742843
  2.8399775 2.8166993 2.6784294]]

 

course: BA in Archeology 
 Neighbours are [[b'BA in Archeology' b'BSc in Botanical Sciences'
  b'BSc in Industrial Engineering'
  b'BSc Geology and Geotherrmal Resources' b'BSc with Geology'
  b'BSc in Mathematics and Statistics'
  b'Bachelor of Commerce in Banking and Financial Services '
  b'Mining Engineering' b'Bachelor of Commerce in Accounting'
  b'BSc with Computer Science']] 
 values are [[7.602413

In [45]:
index.save('model')

In [49]:
!zip -r model.zip model

updating: model/ (stored 0%)
  adding: model/saved_model.pb (deflated 87%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 30%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 30%)
  adding: model/keras_metadata.pb (deflated 84%)
  adding: model/assets/ (stored 0%)
